In [1]:
!mkdir input
!mkdir checkpoint

mkdir: cannot create directory ‘input’: File exists
mkdir: cannot create directory ‘checkpoint’: File exists


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StringType,DoubleType
from pyspark.sql.functions import col

In [3]:
#zdefiniujemy schemat danych
spark = SparkSession.builder \
    .appName("CSVFileStreaming") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/12 13:39:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#definiujemy schemat danych
schema = StructType() \
    .add("order_id",StringType()) \
    .add("category",StringType()) \
    .add("amount",DoubleType())

In [5]:
#wczytanie danych strumieniowych z folderu
df = spark.readStream \
    .option("sep",",") \
    .schema(schema) \
    .csv("input/") #katolog który monitoruje spark

In [6]:
#agregacja: suma wartości na kategorię
agg_df = df.groupBy("category").sum("amount")

In [7]:
#wyświetlenie wyników na konsoli
query = agg_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("checkpointLocation","checkpoint/") \
    .start()

query.awaitTermination()

-------------------------------------------
Batch: 0
-------------------------------------------
+-----------+------------------+
|   category|       sum(amount)|
+-----------+------------------+
|      books|45.489999999999995|
|electronics|            548.99|
|   category|              NULL|
+-----------+------------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----------+-----------+
|   category|sum(amount)|
+-----------+-----------+
|      books|     117.49|
|electronics|     818.99|
|   category|       NULL|
|   clothing|      134.9|
+-----------+-----------+



25/05/12 13:39:06 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/12 13:42:33 WARN HDFSBackedStateStoreProvider: Error doing snapshots for HDFSStateStoreProvider[id = (op=0,part=112),dir = file:/data/notebook_files/checkpoint/state/0/112]
java.io.IOException: Invalid directory or I/O error occurred for dir: /data/notebook_files/checkpoint/state/0/112
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1461)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.DelegateToFileSystem.listStatus(DelegateToFileSystem.java:177)
	at org.apache.hadoop.fs.ChecksumFs.listStatus(ChecksumFs.java:548)
	at org.apache.hadoop.fs.FileContext$Util$1.next(FileContext.java:1915)
	at org.apache.hadoop.fs.FileContext$Util$1.next(FileContext.java:1911)
	at org.apache.hadoop.fs.FSLinkResolver.resolve(FSLinkResolver.java:90)
	at org.apache.hadoop.fs.FileContext$Util.listSt

StreamingQueryException: StreamingQueryException: [STREAM_FAILED] Query [id = d8580429-8f00-46d4-84a5-1dec9c0d1642, runId = b557ce12-67a2-4578-ae22-cc4c159a0777] terminated with exception: Job aborted due to stage failure: Task 1 in stage 5.0 failed 1 times, most recent failure: Lost task 1.0 in stage 5.0 (TID 405) (ip-10-0-193-147 executor driver): java.lang.IllegalStateException: Error committing version 3 into HDFSStateStore[id=(op=0,part=1),dir=file:/data/notebook_files/checkpoint/state/0/1]
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider$HDFSBackedStateStore.commit(HDFSBackedStateStoreProvider.scala:148)
	at org.apache.spark.sql.execution.streaming.state.StreamingAggregationStateManagerBaseImpl.commit(StreamingAggregationStateManager.scala:90)
	at org.apache.spark.sql.execution.streaming.StateStoreSaveExec.$anonfun$doExecute$7(statefulOperators.scala:511)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark